<a href="https://colab.research.google.com/github/mrasw/Big-Data/blob/main/ets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz   
!tar xf spark-3.3.0-bin-hadoop2.tgz   
!pip install -q findspark

In [3]:
!pip install -U --upgrade pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=d71ea721bd594d6a973751ac00f69eff1a415d3ff7ab7579ede2864a0351a079
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [4]:
import pyspark

print(pyspark.__version__)

!export  | grep SPARK_HOME

3.4.0


In [5]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.pandas as ps
import pandas as pd

/usr/local/lib/python3.9/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [9]:
spark = SparkSession.builder.appName("Streaming Process Files") \
    .config("spark.streaming.stopGracefullyOnShutdown", True).master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

# spark = SparkSession.builder.appName("coba").getOrCreate()

In [27]:
!ls

gdrive	sample_data  spark-3.3.0-bin-hadoop2  spark-3.3.0-bin-hadoop2.tgz


In [51]:
spark.conf.set("spark.sql.streaming.schemaInference", True)

In [62]:
streaming_df = spark.readStream\
    .format("json") \
    .option("cleanSource", "archive") \
    .option("sourceArchiveDir", "/content/gdrive/My Drive/ets_bigdat") \
    .option("maxFilesPerTrigger", 1) \
    .load("/content/gdrive/My Drive/datasets/news/*.json")

In [67]:
# To the schema of the data, place a sample json file and change readStream to read 
streaming_df.printSchema()
print(streaming_df)

root
 |-- authors: string (nullable = true)
 |-- category: string (nullable = true)
 |-- date: string (nullable = true)
 |-- headline: string (nullable = true)
 |-- link: string (nullable = true)
 |-- short_description: string (nullable = true)

DataFrame[authors: string, category: string, date: string, headline: string, link: string, short_description: string]


In [70]:
from pyspark.sql.functions import explode, col

exploded_df = streaming_df \
    .select("authors", "category", "date", "headline", "link", "short_description") \
    .drop("data")

In [72]:
exploded_df.printSchema()
print(exploded_df)

root
 |-- authors: string (nullable = true)
 |-- category: string (nullable = true)
 |-- date: string (nullable = true)
 |-- headline: string (nullable = true)
 |-- link: string (nullable = true)
 |-- short_description: string (nullable = true)



AnalysisException: ignored